In [8]:
## Der nachfolgende Programmcode kann erst vollständig ausgeführt werden, wenn  die Datenaufbereitung für alle Verfahren
#  erfolgt ist. Hierzu müssen zuerst die weiteren als Jupyter Notebook zur Verfügung gestellten Programme vollständig
#  ausgeführt werden.


In [1]:
from multiprocessing import Pool, cpu_count
import gc; gc.enable()
import pandas as pd
import numpy as np
from sklearn import *
import sklearn
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

<h1> 1. Modelling - 
    SUM MIN MAX (ohne Rohdaten)
    

In [3]:
# loading library
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import log_loss

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [4]:
# reference: https://machinelearningmastery.com/how-to-save-a-numpy-array-to-file-for-machine-learning/
X_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_train.npz")
y_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_wsdm_labels.npz")
X_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_cv.npz")
y_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_cv_labels.npz")


In [5]:
X_train = X_train['arr_0']
X_test = X_test['arr_0']
y_train = y_train['arr_0']
y_test = y_test['arr_0']

# getting shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(314428, 37) (314428,)
(134756, 37) (134756,)


<h2> Neuronales Netz

In [22]:
# getting device information
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7163838283277888506]

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Flatten, Concatenate, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
init = tf.keras.initializers.HeUniform()
regular = tf.keras.regularizers.l2(l2=0.01)

# Input layer
input_layer = Input(shape=(37,))

# Dense hidden layer 1
layer1 = Dense(64, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(input_layer)

# Batch Normalization layer 1
b_norm1 = BatchNormalization()(layer1)

# Dropout layer 1
drop1 = Dropout(rate=0.5)(b_norm1)

# Dense hidden layer 2
layer2 = Dense(32, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(drop1)

# Batch Normalization layer 2
b_norm2 = BatchNormalization()(layer2)

# Dropout layer 2
drop2 = Dropout(rate=0.5)(b_norm2)

# Dense hidden layer 3
layer3 = Dense(16, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(drop2)

# Batch Normalization layer 3
b_norm3 = BatchNormalization()(layer3)

# Dropout layer 3
drop3 = Dropout(rate=0.5)(b_norm3)

# Output layer
output_layer = Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.glorot_uniform(), kernel_regularizer=regular)(drop3)

# Creating a model
model = Model(inputs=input_layer, outputs=output_layer)

# summary
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 37)]              0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                2432      
_________________________________________________________________
batch_normalization_6 (Batch (None, 64)                256       
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 32)                2080      
_________________________________________________________________
batch_normalization_7 (Batch (None, 32)                128       
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0   

In [25]:
# Compile model
adam = tf.keras.optimizers.Adam()
## Verwende Loss Funktion
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['Accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [26]:
filepath = r"D:\Work_Masterarbeit\wsdm\preprocessed_2\best_model.h5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

es = EarlyStopping(monitor='val_loss', patience=3, mode='min', verbose=1, restore_best_weights=True)

cb = [es, checkpoint]

In [27]:
# Fit the model
history = model.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_test, y_test), verbose=1, callbacks=cb)

Epoch 1/10
1229/1229 [==============================] - 5s 3ms/step - loss: 0.7622 - Accuracy: 0.8799 - recall_1: 0.4330 - precision_1: 0.3600 - val_loss: 0.2246 - val_Accuracy: 0.9387 - val_recall_1: 0.4451 - val_precision_1: 0.8021

Epoch 00001: val_loss improved from inf to 0.22464, saving model to D:\Work_Masterarbeit\wsdm\preprocessed_2\best_model.h5
Epoch 2/10
1229/1229 [==============================] - 4s 3ms/step - loss: 0.2121 - Accuracy: 0.9375 - recall_1: 0.4823 - precision_1: 0.7304 - val_loss: 0.2361 - val_Accuracy: 0.9420 - val_recall_1: 0.6632 - val_precision_1: 0.6945

Epoch 00002: val_loss did not improve from 0.22464
Epoch 3/10
1229/1229 [==============================] - 4s 3ms/step - loss: 0.1907 - Accuracy: 0.9398 - recall_1: 0.4987 - precision_1: 0.7471 - val_loss: 0.1739 - val_Accuracy: 0.9453 - val_recall_1: 0.5479 - val_precision_1: 0.7952

Epoch 00003: val_loss improved from 0.22464 to 0.17394, saving model to D:\Work_Masterarbeit\wsdm\preprocessed_2\best_mod

<h1> 2. Modelling - 
    SUM MIN MAX (inkl. Rohdaten)
    

In [129]:
# loading library
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import log_loss

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [130]:
# reference: https://machinelearningmastery.com/how-to-save-a-numpy-array-to-file-for-machine-learning/
X_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_train_incraw.npz")
y_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_wsdm_labels_incraw.npz")
X_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_cv_incraw.npz")
y_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_cv_labels_incraw.npz")


In [131]:
X_train = X_train['arr_0']
X_test = X_test['arr_0']
y_train = y_train['arr_0']
y_test = y_test['arr_0']

# getting shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4137529, 52) (4137529,)
(1773228, 52) (1773228,)


<h2> Neuronales Netz

In [132]:
# getting device information
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11663723608212709350]

In [133]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Flatten, Concatenate, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
init = tf.keras.initializers.HeUniform()
regular = tf.keras.regularizers.l2(l2=0.01)

# Input layer
input_layer = Input(shape=(52,))

# Dense hidden layer 1
layer1 = Dense(64, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(input_layer)

# Batch Normalization layer 1
b_norm1 = BatchNormalization()(layer1)

# Dropout layer 1
drop1 = Dropout(rate=0.5)(b_norm1)

# Dense hidden layer 2
layer2 = Dense(32, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(drop1)

# Batch Normalization layer 2
b_norm2 = BatchNormalization()(layer2)

# Dropout layer 2
drop2 = Dropout(rate=0.5)(b_norm2)

# Dense hidden layer 3
layer3 = Dense(16, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(drop2)

# Batch Normalization layer 3
b_norm3 = BatchNormalization()(layer3)

# Dropout layer 3
drop3 = Dropout(rate=0.5)(b_norm3)

# Output layer
output_layer = Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.glorot_uniform(), kernel_regularizer=regular)(drop3)

# Creating a model
model = Model(inputs=input_layer, outputs=output_layer)

# summary
model.summary()

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 52)]              0         
_________________________________________________________________
dense_60 (Dense)             (None, 64)                3392      
_________________________________________________________________
batch_normalization_45 (Batc (None, 64)                256       
_________________________________________________________________
dropout_45 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_61 (Dense)             (None, 32)                2080      
_________________________________________________________________
batch_normalization_46 (Batc (None, 32)                128       
_________________________________________________________________
dropout_46 (Dropout)         (None, 32)                0  

In [135]:
# Compile model
adam = tf.keras.optimizers.Adam()
## Verwende Loss Funktion
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['Accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [136]:
filepath = r"D:\Work_Masterarbeit\wsdm\preprocessed_2\best_model.h5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

es = EarlyStopping(monitor='val_loss', patience=3, mode='min', verbose=1, restore_best_weights=True)

cb = [es, checkpoint]

In [137]:
# Fit the model
history = model.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_test, y_test), verbose=1, callbacks=cb)

Epoch 1/10
16163/16163 [==============================] - 43s 3ms/step - loss: 0.2816 - Accuracy: 0.9143 - recall_14: 0.3337 - precision_14: 0.6659 - val_loss: 0.2519 - val_Accuracy: 0.9128 - val_recall_14: 0.1609 - val_precision_14: 0.9238

Epoch 00001: val_loss improved from inf to 0.25188, saving model to D:\Work_Masterarbeit\wsdm\preprocessed_2\best_model.h5
Epoch 2/10
16163/16163 [==============================] - 40s 2ms/step - loss: 0.2238 - Accuracy: 0.9194 - recall_14: 0.3514 - precision_14: 0.7219 - val_loss: 0.4553 - val_Accuracy: 0.8961 - val_recall_14: 0.5481 - val_precision_14: 0.4932

Epoch 00002: val_loss did not improve from 0.25188
Epoch 3/10
16163/16163 [==============================] - 40s 2ms/step - loss: 0.2233 - Accuracy: 0.9194 - recall_14: 0.3526 - precision_14: 0.7212 - val_loss: 0.2083 - val_Accuracy: 0.9292 - val_recall_14: 0.4031 - val_precision_14: 0.8094

Epoch 00003: val_loss improved from 0.25188 to 0.20832, saving model to D:\Work_Masterarbeit\wsdm\pr

<h1> 3. Modelling - 
    Domain Driven Feature Labeling (ohne Rohdaten)
    

In [120]:
# loading library
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import log_loss

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [121]:
X_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_train.npz")
y_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_wsdm_labels.npz")
X_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_test.npz")
y_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_test_labels.npz")

In [122]:
X_train = X_train['arr_0']
X_test = X_test['arr_0']
y_train = y_train['arr_0']
y_test = y_test['arr_0']

# getting shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(679672, 134) (679672,)
(291288, 134) (291288,)


<h2> Neuronales Netz

In [123]:
# getting device information
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16372060046434871542]

In [124]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Flatten, Concatenate, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
init = tf.keras.initializers.HeUniform()
regular = tf.keras.regularizers.l2(l2=0.01)

# Input layer
input_layer = Input(shape=(134,))

# Dense hidden layer 1
layer1 = Dense(64, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(input_layer)

# Batch Normalization layer 1
b_norm1 = BatchNormalization()(layer1)

# Dropout layer 1
drop1 = Dropout(rate=0.5)(b_norm1)

# Dense hidden layer 2
layer2 = Dense(32, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(drop1)

# Batch Normalization layer 2
b_norm2 = BatchNormalization()(layer2)

# Dropout layer 2
drop2 = Dropout(rate=0.5)(b_norm2)

# Dense hidden layer 3
layer3 = Dense(16, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(drop2)

# Batch Normalization layer 3
b_norm3 = BatchNormalization()(layer3)

# Dropout layer 3
drop3 = Dropout(rate=0.5)(b_norm3)

# Output layer
output_layer = Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.glorot_uniform(), kernel_regularizer=regular)(drop3)

# Creating a model
model = Model(inputs=input_layer, outputs=output_layer)

# summary
model.summary()

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 134)]             0         
_________________________________________________________________
dense_56 (Dense)             (None, 64)                8640      
_________________________________________________________________
batch_normalization_42 (Batc (None, 64)                256       
_________________________________________________________________
dropout_42 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_57 (Dense)             (None, 32)                2080      
_________________________________________________________________
batch_normalization_43 (Batc (None, 32)                128       
_________________________________________________________________
dropout_43 (Dropout)         (None, 32)                0  

In [126]:
# Compile model
adam = tf.keras.optimizers.Adam()
## Verwende Loss Funktion
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['Accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [127]:
filepath = r"D:\Work_Masterarbeit\wsdm\preprocessed_4\best_modelddl.h5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

es = EarlyStopping(monitor='val_loss', patience=3, mode='min', verbose=1, restore_best_weights=True)

cb = [es, checkpoint]

In [128]:
# Fit the model
history = model.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_test, y_test), verbose=1, callbacks=cb)

Epoch 1/10
2655/2655 [==============================] - 9s 3ms/step - loss: 0.4084 - Accuracy: 0.9304 - recall_13: 0.5044 - precision_13: 0.6452 - val_loss: 0.3106 - val_Accuracy: 0.9521 - val_recall_13: 0.5632 - val_precision_13: 0.8518

Epoch 00001: val_loss improved from inf to 0.31056, saving model to D:\Work_Masterarbeit\wsdm\preprocessed_4\best_modelddl.h5
Epoch 2/10
2655/2655 [==============================] - 7s 3ms/step - loss: 0.1791 - Accuracy: 0.9501 - recall_13: 0.5708 - precision_13: 0.8210 - val_loss: 0.3708 - val_Accuracy: 0.9212 - val_recall_13: 0.7491 - val_precision_13: 0.5439

Epoch 00002: val_loss did not improve from 0.31056
Epoch 3/10
2655/2655 [==============================] - 7s 3ms/step - loss: 0.1738 - Accuracy: 0.9500 - recall_13: 0.5703 - precision_13: 0.8194 - val_loss: 0.2281 - val_Accuracy: 0.9334 - val_recall_13: 0.2654 - val_precision_13: 0.9681

Epoch 00003: val_loss improved from 0.31056 to 0.22812, saving model to D:\Work_Masterarbeit\wsdm\preproce

<h1> 4. Modelling - 
    Domain Driven Feature Labeling (mit Rohdaten)
    

In [2]:
# loading library
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import log_loss

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [3]:
X_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_train_incraw.npz")
y_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_wsdm_labels_incraw.npz")
X_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_test_incraw.npz")
y_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_test_labels_incraw.npz")

In [4]:
X_train = X_train['arr_0']
X_test = X_test['arr_0']
y_train = y_train['arr_0']
y_test = y_test['arr_0']

# getting shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(3310024, 160) (3310024,)
(1418582, 160) (1418582,)


<h2> Neuronales Netz

In [5]:
# getting device information
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 235336893697470730]

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Flatten, Concatenate, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
init = tf.keras.initializers.HeUniform()
regular = tf.keras.regularizers.l2(l2=0.01)

# Input layer
input_layer = Input(shape=(160,))

# Dense hidden layer 1
layer1 = Dense(64, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(input_layer)

# Batch Normalization layer 1
b_norm1 = BatchNormalization()(layer1)

# Dropout layer 1
drop1 = Dropout(rate=0.5)(b_norm1)

# Dense hidden layer 2
layer2 = Dense(32, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(drop1)

# Batch Normalization layer 2
b_norm2 = BatchNormalization()(layer2)

# Dropout layer 2
drop2 = Dropout(rate=0.5)(b_norm2)

# Dense hidden layer 3
layer3 = Dense(16, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(drop2)

# Batch Normalization layer 3
b_norm3 = BatchNormalization()(layer3)

# Dropout layer 3
drop3 = Dropout(rate=0.5)(b_norm3)

# Output layer
output_layer = Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.glorot_uniform(), kernel_regularizer=regular)(drop3)

# Creating a model
model = Model(inputs=input_layer, outputs=output_layer)

# summary
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 160)]             0         
_________________________________________________________________
dense (Dense)                (None, 64)                10304     
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0     

In [7]:
# Compile model
adam = tf.keras.optimizers.Adam()
## Verwende Loss Funktion
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['Accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [8]:
filepath = r"D:\Work_Masterarbeit\wsdm\preprocessed_4\best_modelddl.h5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

es = EarlyStopping(monitor='val_loss', patience=3, mode='min', verbose=1, restore_best_weights=True)

cb = [es, checkpoint]

In [9]:
# Fit the model
history = model.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_test, y_test), verbose=1, callbacks=cb)

Epoch 1/10
12930/12930 [==============================] - 31s 2ms/step - loss: 0.3710 - Accuracy: 0.8937 - recall: 0.0135 - precision: 0.2186 - val_loss: 0.3438 - val_Accuracy: 0.8971 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00

Epoch 00001: val_loss improved from inf to 0.34376, saving model to D:\Work_Masterarbeit\wsdm\preprocessed_4\best_modelddl.h5
Epoch 2/10
12930/12930 [==============================] - 30s 2ms/step - loss: 0.3203 - Accuracy: 0.8967 - recall: 0.0142 - precision: 0.4244 - val_loss: 0.3709 - val_Accuracy: 0.8971 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00

Epoch 00002: val_loss did not improve from 0.34376
Epoch 3/10
12930/12930 [==============================] - 29s 2ms/step - loss: 0.3221 - Accuracy: 0.8968 - recall: 0.0057 - precision: 0.3530 - val_loss: 0.3463 - val_Accuracy: 0.8971 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00

Epoch 00003: val_loss did not improve from 0.34376
Epoch 4/10
12930/12930 [==============================] - 

<h1> 5. Modelling -  Rohdaten
    

In [55]:
# loading library
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import log_loss

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [56]:
X_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\actual_train.npz")
y_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\actual_wsdm_labels.npz")
X_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\actual_cv.npz")
y_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_3\actual_cv_labels.npz")

In [57]:
X_train = X_train['arr_0']
X_test = X_test['arr_0']
y_train = y_train['arr_0']
y_test = y_test['arr_0']

# getting shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5910756, 21) (5910756,)
(2533182, 21) (2533182,)


<h2> Neuronales Netz

In [65]:
# getting device information
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16840122965060518509]

In [66]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Flatten, Concatenate, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
init = tf.keras.initializers.HeUniform()
regular = tf.keras.regularizers.l2(l2=0.01)

# Input layer
input_layer = Input(shape=(21,))

# Dense hidden layer 1
layer1 = Dense(64, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(input_layer)

# Batch Normalization layer 1
b_norm1 = BatchNormalization()(layer1)

# Dropout layer 1
drop1 = Dropout(rate=0.5)(b_norm1)

# Dense hidden layer 2
layer2 = Dense(32, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(drop1)

# Batch Normalization layer 2
b_norm2 = BatchNormalization()(layer2)

# Dropout layer 2
drop2 = Dropout(rate=0.5)(b_norm2)

# Dense hidden layer 3
layer3 = Dense(16, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(drop2)

# Batch Normalization layer 3
b_norm3 = BatchNormalization()(layer3)

# Dropout layer 3
drop3 = Dropout(rate=0.5)(b_norm3)

# Output layer
output_layer = Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.glorot_uniform(), kernel_regularizer=regular)(drop3)

# Creating a model
model = Model(inputs=input_layer, outputs=output_layer)

# summary
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 21)]              0         
_________________________________________________________________
dense_28 (Dense)             (None, 64)                1408      
_________________________________________________________________
batch_normalization_21 (Batc (None, 64)                256       
_________________________________________________________________
dropout_21 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 32)                2080      
_________________________________________________________________
batch_normalization_22 (Batc (None, 32)                128       
_________________________________________________________________
dropout_22 (Dropout)         (None, 32)                0   

In [68]:
# Compile model
adam = tf.keras.optimizers.Adam()
## Verwende Loss Funktion
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['Accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [69]:
filepath = r"D:\Work_Masterarbeit\wsdm\preprocessed_4\best_modelddl.h5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

es = EarlyStopping(monitor='val_loss', patience=3, mode='min', verbose=1, restore_best_weights=True)

cb = [es, checkpoint]

In [70]:
# Fit the model
history = model.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_test, y_test), verbose=1, callbacks=cb)

Epoch 1/10
23089/23089 [==============================] - 53s 2ms/step - loss: 0.2945 - Accuracy: 0.9123 - recall_6: 0.2929 - precision_6: 0.6691 - val_loss: 0.2574 - val_Accuracy: 0.9194 - val_recall_6: 0.2939 - val_precision_6: 0.7888

Epoch 00001: val_loss improved from inf to 0.25737, saving model to D:\Work_Masterarbeit\wsdm\preprocessed_4\best_modelddl.h5
Epoch 2/10
23089/23089 [==============================] - 60s 3ms/step - loss: 0.2531 - Accuracy: 0.9145 - recall_6: 0.2864 - precision_6: 0.7112 - val_loss: 0.2440 - val_Accuracy: 0.9194 - val_recall_6: 0.3002 - val_precision_6: 0.7803

Epoch 00002: val_loss improved from 0.25737 to 0.24402, saving model to D:\Work_Masterarbeit\wsdm\preprocessed_4\best_modelddl.h5
Epoch 3/10
23089/23089 [==============================] - 57s 2ms/step - loss: 0.2533 - Accuracy: 0.9146 - recall_6: 0.2861 - precision_6: 0.7131 - val_loss: 0.2362 - val_Accuracy: 0.9197 - val_recall_6: 0.2692 - val_precision_6: 0.8417

Epoch 00003: val_loss improved

<h1> 6. Modelling -  Manuelles Feature Engineering (Top 10% Kaggle Leaderboard)
    

In [112]:
# loading library
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import log_loss

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [113]:
# reference: https://machinelearningmastery.com/how-to-save-a-numpy-array-to-file-for-machine-learning/
X_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_train.npz")
y_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_wsdm_labels.npz")
X_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_cv.npz")
y_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_cv_labels.npz")

In [114]:
X_train = X_train['arr_0']
X_test = X_test['arr_0']
y_train = y_train['arr_0']
y_test = y_test['arr_0']

# getting shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(339836, 57) (339836,)
(145644, 57) (145644,)


<h2> Neuronales Netz

In [115]:
# getting device information
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4121773300898880399]

In [116]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Flatten, Concatenate, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
init = tf.keras.initializers.HeUniform()
regular = tf.keras.regularizers.l2(l2=0.01)

# Input layer
input_layer = Input(shape=(57,))

# Dense hidden layer 1
layer1 = Dense(64, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(input_layer)

# Batch Normalization layer 1
b_norm1 = BatchNormalization()(layer1)

# Dropout layer 1
drop1 = Dropout(rate=0.5)(b_norm1)

# Dense hidden layer 2
layer2 = Dense(32, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(drop1)

# Batch Normalization layer 2
b_norm2 = BatchNormalization()(layer2)

# Dropout layer 2
drop2 = Dropout(rate=0.5)(b_norm2)

# Dense hidden layer 3
layer3 = Dense(16, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(drop2)

# Batch Normalization layer 3
b_norm3 = BatchNormalization()(layer3)

# Dropout layer 3
drop3 = Dropout(rate=0.5)(b_norm3)

# Output layer
output_layer = Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.glorot_uniform(), kernel_regularizer=regular)(drop3)

# Creating a model
model = Model(inputs=input_layer, outputs=output_layer)

# summary
model.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 57)]              0         
_________________________________________________________________
dense_52 (Dense)             (None, 64)                3712      
_________________________________________________________________
batch_normalization_39 (Batc (None, 64)                256       
_________________________________________________________________
dropout_39 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_53 (Dense)             (None, 32)                2080      
_________________________________________________________________
batch_normalization_40 (Batc (None, 32)                128       
_________________________________________________________________
dropout_40 (Dropout)         (None, 32)                0  

In [117]:
# Compile model
adam = tf.keras.optimizers.Adam()
## Verwende Loss Funktion
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['Accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [118]:
filepath = r"D:\Work_Masterarbeit\wsdm\preprocessed_4\best_modelddl.h5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

es = EarlyStopping(monitor='val_loss', patience=3, mode='min', verbose=1, restore_best_weights=True)

cb = [es, checkpoint]

In [119]:
# Fit the model
history = model.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_test, y_test), verbose=1, callbacks=cb)

Epoch 1/10
1328/1328 [==============================] - 6s 3ms/step - loss: 0.6754 - Accuracy: 0.8871 - recall_12: 0.5313 - precision_12: 0.4046 - val_loss: 0.2532 - val_Accuracy: 0.9555 - val_recall_12: 0.6320 - val_precision_12: 0.8367

Epoch 00001: val_loss improved from inf to 0.25320, saving model to D:\Work_Masterarbeit\wsdm\preprocessed_4\best_modelddl.h5
Epoch 2/10
1328/1328 [==============================] - 3s 2ms/step - loss: 0.1847 - Accuracy: 0.9510 - recall_12: 0.6378 - precision_12: 0.7795 - val_loss: 0.3063 - val_Accuracy: 0.9004 - val_recall_12: 0.8915 - val_precision_12: 0.4738

Epoch 00002: val_loss did not improve from 0.25320
Epoch 3/10
1328/1328 [==============================] - 4s 3ms/step - loss: 0.1669 - Accuracy: 0.9529 - recall_12: 0.6615 - precision_12: 0.7835 - val_loss: 0.1447 - val_Accuracy: 0.9553 - val_recall_12: 0.7242 - val_precision_12: 0.7687

Epoch 00003: val_loss improved from 0.25320 to 0.14473, saving model to D:\Work_Masterarbeit\wsdm\preproce

<h1> 7. Modelling -  Manuelles Feature Engineering (Top 10% Kaggle Leaderboard) mit Rohdaten
    

In [104]:
# loading library
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import log_loss

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [105]:
# reference: https://machinelearningmastery.com/how-to-save-a-numpy-array-to-file-for-machine-learning/
X_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_train_incraw.npz")
y_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_wsdm_labels_incraw.npz")
X_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_cv_incraw.npz")
y_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed\actual_cv_labels_incraw.npz")

In [106]:
X_train = X_train['arr_0']
X_test = X_test['arr_0']
y_train = y_train['arr_0']
y_test = y_test['arr_0']

# getting shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5910756, 68) (5910756,)
(2533182, 68) (2533182,)


<h2> Neuronales Netz

In [107]:
# getting device information
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8984606731326651300]

In [108]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Flatten, Concatenate, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
init = tf.keras.initializers.HeUniform()
regular = tf.keras.regularizers.l2(l2=0.01)

# Input layer
input_layer = Input(shape=(68,))

# Dense hidden layer 1
layer1 = Dense(64, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(input_layer)

# Batch Normalization layer 1
b_norm1 = BatchNormalization()(layer1)

# Dropout layer 1
drop1 = Dropout(rate=0.5)(b_norm1)

# Dense hidden layer 2
layer2 = Dense(32, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(drop1)

# Batch Normalization layer 2
b_norm2 = BatchNormalization()(layer2)

# Dropout layer 2
drop2 = Dropout(rate=0.5)(b_norm2)

# Dense hidden layer 3
layer3 = Dense(16, activation='relu', kernel_initializer=init, kernel_regularizer=regular)(drop2)

# Batch Normalization layer 3
b_norm3 = BatchNormalization()(layer3)

# Dropout layer 3
drop3 = Dropout(rate=0.5)(b_norm3)

# Output layer
output_layer = Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.glorot_uniform(), kernel_regularizer=regular)(drop3)

# Creating a model
model = Model(inputs=input_layer, outputs=output_layer)

# summary
model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 68)]              0         
_________________________________________________________________
dense_48 (Dense)             (None, 64)                4416      
_________________________________________________________________
batch_normalization_36 (Batc (None, 64)                256       
_________________________________________________________________
dropout_36 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_49 (Dense)             (None, 32)                2080      
_________________________________________________________________
batch_normalization_37 (Batc (None, 32)                128       
_________________________________________________________________
dropout_37 (Dropout)         (None, 32)                0  

In [109]:
# Compile model
adam = tf.keras.optimizers.Adam()
## Verwende Loss Funktion
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['Accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [110]:
filepath = r"D:\Work_Masterarbeit\wsdm\preprocessed_4\best_modelddl.h5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

es = EarlyStopping(monitor='val_loss', patience=3, mode='min', verbose=1, restore_best_weights=True)

cb = [es, checkpoint]

In [111]:
# Fit the model
history = model.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_test, y_test), verbose=1, callbacks=cb)

Epoch 1/10
23089/23089 [==============================] - 64s 3ms/step - loss: 0.2514 - Accuracy: 0.9229 - recall_11: 0.4346 - precision_11: 0.7022 - val_loss: 0.1993 - val_Accuracy: 0.9347 - val_recall_11: 0.5187 - val_precision_11: 0.7709

Epoch 00001: val_loss improved from inf to 0.19926, saving model to D:\Work_Masterarbeit\wsdm\preprocessed_4\best_modelddl.h5
Epoch 2/10
23089/23089 [==============================] - 57s 2ms/step - loss: 0.2112 - Accuracy: 0.9260 - recall_11: 0.4540 - precision_11: 0.7236 - val_loss: 0.1829 - val_Accuracy: 0.9366 - val_recall_11: 0.6352 - val_precision_11: 0.7157

Epoch 00002: val_loss improved from 0.19926 to 0.18289, saving model to D:\Work_Masterarbeit\wsdm\preprocessed_4\best_modelddl.h5
Epoch 3/10
23089/23089 [==============================] - 57s 2ms/step - loss: 0.2107 - Accuracy: 0.9262 - recall_11: 0.4577 - precision_11: 0.7229 - val_loss: 0.1799 - val_Accuracy: 0.9371 - val_recall_11: 0.5273 - val_precision_11: 0.7905

Epoch 00003: val_l